In [259]:
import sys
sys.path.append("utils/")
import wv_util as wv
from utils.dataset_utils import *

In [260]:
coords1, chips1, classes1 = wv.get_labels('/data/zjc4/xView_train.geojson')

100%|██████████| 601937/601937 [00:04<00:00, 146150.27it/s]


In [261]:
idxs = np.arange(0,coords1.shape[0]).reshape(-1,1)
coords1_idx = np.hstack( (coords1, idxs ))


In [262]:
coords1_idx.shape

(601937, 5)

In [263]:
import aug_util as aug
import wv_util as wv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import tqdm
import pickle
import itertools
import cv2 as cv
import glob
import random
import os
import imp

from PIL import Image
from sklearn.model_selection import train_test_split
import dataset_utils as du

imp.reload(wv)
imp.reload(du)

import random
s = 180
#s = 15
#s = 30
np.random.seed(s)
random.seed(s)
unique_chips = np.unique(chips1)
selected_images = np.random.choice(unique_chips,440)
idxs = np.isin(chips1,selected_images)

# Input desired classes
grouped_classes = [[77,73],[11,12],[13],[17,18,20,21],
       [19,23,24,25,28,29,60,61,65,26],[41,42,50,40,44,45,47,49]]

labels = ["building and facility" ,"small aircraft", 
          "large aircraft","vehicles","bus","boat"]

xdataset = du.XviewDataset(grouped_classes,labels, coords1_idx[idxs],
                           chips1[idxs],classes1[idxs])

tif_names,distrbs = xdataset.splitTrainTest(debug=True)
dfs,gc = distrbs

string_sets = ["train","valid"]
datum = list(zip(string_sets,tif_names))

In [274]:
temp_df = dfs[0]
temp_df['count'] = pd.to_numeric(temp_df['count'])
print(temp_df[['grouped cls','count']].groupby(['grouped cls']).sum())

                          count
grouped cls                    
boat                     2019.0
building and facility  109452.0
bus                     10900.0
large aircraft            217.0
small aircraft            156.0
vehicles                64087.0


In [263]:
temp_df = dfs[1]
temp_df['count'] = pd.to_numeric(temp_df['count'])
temp_df[['grouped cls','count']].groupby(['grouped cls']).sum()

346it [00:05, 66.83it/s]


,count
grouped cls,
boat,387.0
building and facility,23241.0
bus,2537.0
large aircraft,98.0
small aircraft,24.0
vehicles,14719.0


In [264]:
"""
Generate darknet images for 30 cm
"""
# Deleting all old images
os.system("rm /data/zjc4/chipped-30/data/images/*")
os.system("rm /data/zjc4/chipped-30/data/labels/*")
os.system("rm /data/zjc4/chipped-30/data/xview_img_train.txt")
os.system("rm /data/zjc4/chipped-30/data/xview_img_valid.txt")
os.system("rm /data/zjc4/chipped-30/data/xview_img_test.txt")

dnf = du.DarkNetFormatter(output_dir_ = "/data/zjc4/chipped-30/data/",
                       input_dir_="/data/zjc4/",
                     coords_ = coords1_idx[idxs],
                       chips_ = chips1[idxs],
                       classes_ = classes1[idxs],
                       grouped_classes_=grouped_classes)
dnf.transformDatum(datum,30,showImg=False,chipImage=True)

[Errno 2] No such file or directory: '/data/zjc4/train_images/1395.tif'
/data/zjc4/chipped-30/data/xview_img_train.txt
/data/zjc4/chipped-30/data/xview_img_valid.txt


In [269]:
import dataset_utils as du
imp.reload(du)

"""
Generate darknet images for 80 cm
"""
# Deleting all old images
os.system("mkdir /data/zjc4/chipped-80/")
os.system("mkdir /data/zjc4/chipped-80/data/")
os.system("mkdir /data/zjc4/chipped-80/data/images/")
os.system("mkdir /data/zjc4/chipped-80/data/labels/")
os.system("rm /data/zjc4/chipped-80/data/images/*")
os.system("rm /data/zjc4/chipped-80/data/labels/*")
os.system("rm /data/zjc4/chipped-80/data/xview_img_train.txt")
os.system("rm /data/zjc4/chipped-80/data/xview_img_valid.txt")
os.system("rm /data/zjc4/chipped-80/data/xview_img_test.txt")

dnf2 = du.DarkNetFormatter(output_dir_ = "/data/zjc4/chipped-80/data/",
                       input_dir_="/data/zjc4/",
                       coords_ = coords1_idx[idxs],
                       chips_ = chips1[idxs],
                       classes_ = classes1[idxs],
                       grouped_classes_=grouped_classes)
dnf2.transformDatum(datum,80,showImg=False,chipImage=True)

[Errno 2] No such file or directory: '/data/zjc4/train_images/1395.tif'
/data/zjc4/chipped-80/data/xview_img_train.txt
/data/zjc4/chipped-80/data/xview_img_valid.txt


In [265]:
import dataset_utils as du
imp.reload(du)

"""
Generate darknet images for 100 cm
"""
# Deleting all old images
os.system("mkdir /data/zjc4/chipped-100/")
os.system("mkdir /data/zjc4/chipped-100/data/")
os.system("mkdir /data/zjc4/chipped-100/data/images/")
os.system("mkdir /data/zjc4/chipped-100/data/labels/")
os.system("rm /data/zjc4/chipped-100/data/images/*")
os.system("rm /data/zjc4/chipped-100/data/labels/*")
os.system("rm /data/zjc4/chipped-100/data/xview_img_train.txt")
os.system("rm /data/zjc4/chipped-100/data/xview_img_valid.txt")
os.system("rm /data/zjc4/chipped-100/data/xview_img_test.txt")

dnf2 = du.DarkNetFormatter(output_dir_ = "/data/zjc4/chipped-100/data/",
                       input_dir_="/data/zjc4/",
                       coords_ = coords1_idx[idxs],
                       chips_ = chips1[idxs],
                       classes_ = classes1[idxs],
                       grouped_classes_=grouped_classes)
dnf2.transformDatum(datum,100,showImg=False,chipImage=True)

[Errno 2] No such file or directory: '/data/zjc4/train_images/1395.tif'
/data/zjc4/chipped-100/data/xview_img_train.txt
/data/zjc4/chipped-100/data/xview_img_valid.txt


In [272]:
import dataset_utils as du
imp.reload(du)

"""
Generate darknet images for 200 cm
"""
# Deleting all old images
os.system("mkdir /data/zjc4/chipped-200/")
os.system("mkdir /data/zjc4/chipped-200/data/")
os.system("mkdir /data/zjc4/chipped-200/data/images/")
os.system("mkdir /data/zjc4/chipped-200/data/labels/")
os.system("rm /data/zjc4/chipped-200/data/images/*")
os.system("rm /data/zjc4/chipped-200/data/labels/*")
os.system("rm /data/zjc4/chipped-200/data/xview_img_train.txt")
os.system("rm /data/zjc4/chipped-200/data/xview_img_valid.txt")
os.system("rm /data/zjc4/chipped-200/data/xview_img_test.txt")

dnf2 = du.DarkNetFormatter(output_dir_ = "/data/zjc4/chipped-200/data/",
                       input_dir_="/data/zjc4/",
                       coords_ = coords1_idx[idxs],
                       chips_ = chips1[idxs],
                       classes_ = classes1[idxs],
                       grouped_classes_=grouped_classes)
dnf2.transformDatum(datum,200,showImg=False,chipImage=False)

[Errno 2] No such file or directory: '/data/zjc4/train_images/1395.tif'
/data/zjc4/chipped-200/data/xview_img_train.txt
/data/zjc4/chipped-200/data/xview_img_valid.txt


In [270]:
import dataset_utils as du
imp.reload(du)

"""
Generate darknet images for 200 cm
"""
# Deleting all old images
os.system("mkdir /data/zjc4/chipped-300/")
os.system("mkdir /data/zjc4/chipped-300/data/")
os.system("mkdir /data/zjc4/chipped-300/data/images/")
os.system("mkdir /data/zjc4/chipped-300/data/labels/")
os.system("rm /data/zjc4/chipped-300/data/images/*")
os.system("rm /data/zjc4/chipped-300/data/labels/*")
os.system("rm /data/zjc4/chipped-300/data/xview_img_train.txt")
os.system("rm /data/zjc4/chipped-300/data/xview_img_valid.txt")
os.system("rm /data/zjc4/chipped-300/data/xview_img_test.txt")

dnf2 = du.DarkNetFormatter(output_dir_ = "/data/zjc4/chipped-300/data/",
                       input_dir_="/data/zjc4/",
                       coords_ = coords1_idx[idxs],
                       chips_ = chips1[idxs],
                       classes_ = classes1[idxs],
                       grouped_classes_=grouped_classes)
dnf2.transformDatum(datum,200,showImg=False,chipImage=False)

[Errno 2] No such file or directory: '/data/zjc4/train_images/1395.tif'
/data/zjc4/chipped-200/data/xview_img_train.txt
/data/zjc4/chipped-200/data/xview_img_valid.txt


In [271]:
import dataset_utils as du
imp.reload(du)

"""
Generate darknet images for 1000 cm
"""
# Deleting all old images
os.system("mkdir /data/zjc4/chipped-1000/")
os.system("mkdir /data/zjc4/chipped-1000/data/")
os.system("mkdir /data/zjc4/chipped-1000/data/images/")
os.system("mkdir /data/zjc4/chipped-1000/data/labels/")
os.system("rm /data/zjc4/chipped-1000/data/images/*")
os.system("rm /data/zjc4/chipped-1000/data/labels/*")
os.system("rm /data/zjc4/chipped-1000/data/xview_img_train.txt")
os.system("rm /data/zjc4/chipped-1000/data/xview_img_valid.txt")
os.system("rm /data/zjc4/chipped-1000/data/xview_img_test.txt")

dnf2 = du.DarkNetFormatter(output_dir_ = "/data/zjc4/chipped-1000/data/",
                       input_dir_="/data/zjc4/",
                       coords_ = coords1_idx[idxs],
                       chips_ = chips1[idxs],
                       classes_ = classes1[idxs],
                       grouped_classes_=grouped_classes)
dnf2.transformDatum(datum,1000,showImg=False,chipImage=False)

[Errno 2] No such file or directory: '/data/zjc4/train_images/1395.tif'
/data/zjc4/chipped-1000/data/xview_img_train.txt
/data/zjc4/chipped-1000/data/xview_img_valid.txt


In [ ]:
"""
Generate darknet images for 1000 cm
"""
# Deleting all old images
os.system("rm /data/zjc4/chipped-30/data/images/*")
os.system("rm /data/zjc4/chipped-30/data/labels/*")
os.system("rm /data/zjc4/chipped-30/data/xview_img_train.txt")
os.system("rm /data/zjc4/chipped-30/data/xview_img_valid.txt")
os.system("rm /data/zjc4/chipped-30/data/xview_img_test.txt")

dnf = du.DarkNetFormatter(output_dir_ = "/data/zjc4/chipped-30/data/",
                       input_dir_="/data/zjc4/",
                     coords_ = coords1_idx[idxs],
                       chips_ = chips1[idxs],
                       classes_ = classes1[idxs],
                       grouped_classes_=grouped_classes)
dnf.transformDatum(datum,30,showImg=False,chipImage=True)